In [3]:
!git clone https://github.com/DavidSie/MotionVectorEstimator.git

Cloning into 'MotionVectorEstimator'...


In [ ]:
import numpy as np
import cv2
import sys
import time
import moviepy.editor as mp


def read_video(filename):
    """returns video object with its properties"""
    video_name = filename.split('/')[-1].split('.')[0]
    video = cv2.VideoCapture(filename)
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    videofps = int(video.get(cv2.CAP_PROP_FPS))
    videoframecount = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    # Exit if video not opened.
    if not video.isOpened():
        print("Could not open video")
        sys.exit()

    return (video_name, video, frame_width, frame_height, videofps, videoframecount)

def read_frame(video, skipFrame=0):
    """returns the next frame from the video object, or skip frames"""
    for skip in range(skipFrame+1):
        ok, frame = video.read()

        if not ok:
            print('Cannot read video file')
            break

    return (ok, frame)

def getFrames(filepath, skip=10, count=2, outfolder="testImages", name="frame", frametype="png"):
    """
    Retrives and saves frames from a video
    :param filepath: file path of video
    :param skip: number of frames to skip by
    :param count: how many frames total to save
    :param outfolder: where to save frames
    :param name: frame name
    :return: None
    """
    video_name, video, frame_width, frame_height, videofps, videoframecount = read_video(filepath)

    for x in range(count):
        ok, frame = read_frame(video, skipFrame=skip)
        if ok:
            cv2.imwrite(f"{outfolder}/{name}{x+1}.{frametype}", frame)
        else:
            print("EOF")
            break

def downSampleVideo(filepath, factor=0.5, outfolder="VIDEOS"):
    """Downsamples a video by the factor to reduce the amount of frames in the video"""
    video_name, video, frame_width, frame_height, videofps, videoframecount = read_video(filepath)
    print(frame_height, frame_width)
    clip = mp.VideoFileClip(filepath)
    clip_resized = clip.resize(factor)
    clip_resized.write_videofile(f"{outfolder}/{video_name}_resized{int(frame_height*factor)}.mp4")




if __name__ == "__main__":

    timer = time.time()

    # COMMAND STARTS HERE

    #videopath = "VIDEOS/CityWalk.mp4"
    #downSampleVideo(videopath)
    #videopath = "VIDEOS/HouseTour.mp4"
    #downSampleVideo(videopath)
    
    getFrames('video_menor.mp4')

    #COMMAND ENDS HERE

    totTime = time.time() - timer
    print(f"Done, executed in {totTime:.2f} s")



In [ ]:
import numpy as np
import cv2 as cv
import argparse

cap = cv.VideoCapture('video_menor.mp4')
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0,255,(100,3))
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
while(1):
    ret,frame = cap.read()
    try:
        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    except:
        cv.destroyAllWindows()
        break
    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv.line(mask, (int(a),int(b)),(int(c),int(d)), color[i].tolist(), 2)
        frame = cv.circle(frame,(int(a),int(b)),5,color[i].tolist(),-1)
    img = cv.add(frame,mask)
    cv.imshow('frame',img)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)